In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf

In [2]:
train_df = pd.read_csv('english.csv')
train_df.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3911 entries, 0 to 3910
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   text_id      3911 non-null   object 
 1   full_text    3911 non-null   object 
 2   cohesion     3911 non-null   float64
 3   syntax       3911 non-null   float64
 4   vocabulary   3911 non-null   float64
 5   phraseology  3911 non-null   float64
 6   grammar      3911 non-null   float64
 7   conventions  3911 non-null   float64
dtypes: float64(6), object(2)
memory usage: 244.6+ KB


In [4]:
train_df[['cohesion' , 'syntax' , 'vocabulary' , 'phraseology' , 'grammar' , 'conventions']].describe().T.style.background_gradient(cmap='Blues')

,count,mean,std,min,25%,50%,75%,max
cohesion,3911.000000,3.127077,0.662542,1.000000,2.500000,3.000000,3.500000,5.000000
syntax,3911.000000,3.028254,0.644399,1.000000,2.500000,3.000000,3.500000,5.000000
vocabulary,3911.000000,3.235745,0.583148,1.000000,3.000000,3.000000,3.500000,5.000000
phraseology,3911.000000,3.116850,0.655997,1.000000,2.500000,3.000000,3.500000,5.000000
grammar,3911.000000,3.032856,0.699841,1.000000,2.500000,3.000000,3.500000,5.000000
conventions,3911.000000,3.081053,0.671450,1.000000,2.500000,3.000000,3.500000,5.000000


In [6]:
train_df['full_text'][0]

"I think that students would benefit from learning at home,because they wont have to change and get up early in the morning to shower and do there hair. taking only classes helps them because at there house they'll be pay more attention. they will be comfortable at home.\n\nThe hardest part of school is getting ready. you wake up go brush your teeth and go to your closet and look at your cloths. after you think you picked a outfit u go look in the mirror and youll either not like it or you look and see a stain. Then you'll have to change. with the online classes you can wear anything and stay home and you wont need to stress about what to wear.\n\nmost students usually take showers before school. they either take it before they sleep or when they wake up. some students do both to smell good. that causes them do miss the bus and effects on there lesson time cause they come late to school. when u have online classes u wont need to miss lessons cause you can get everything set up and go t

In [8]:
import re

text = train_df['full_text']

text = text.str.lower()

text = text.apply(lambda x : re.sub("[^a-z]\s","",x) )

text = text.str.replace("#", "")

text = text.apply(lambda x: ' '.join([w for w in x.split() if len(w)>2 and len(w)<8]))

count_words = text.str.findall(r'(\w+)').str.len()
print(count_words.sum())

1064587


In [9]:
most_freq_words = pd.Series(' '.join(text).lower().split()).value_counts()[:25]
text = text.apply(lambda x : " ".join(word for word in x.split() if word not in most_freq_words ))
print(most_freq_words)

count_words = text.str.findall(r'(\w+)').str.len()
print(count_words.sum())

the        52605
and        39550
you        34777
that       28085
they       23406
have       20274
can        18335
for        16994
are        14465
not        13779
because    13357
people     12681
with       11396
will       11039
your       10814
school     10204
their      10095
more        8942
what        8572
get         8204
good        8027
like        7202
time        7129
work        6859
want        6826
dtype: int64
660970


In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

X = train_df['full_text']
y = np.array(train_df.iloc[:,2:])

tokenizer = Tokenizer(num_words = 10000 , split = ' ')
tokenizer.fit_on_texts(X.values)
X = tokenizer.texts_to_sequences(X.values)
X = pad_sequences(X)

In [17]:
from sklearn.model_selection import train_test_splitain_test_split

X_train,X_test,y_train,y_test = train_test_split(X, 
                                                 y, 
                                                 test_size=0.2)

In [13]:
X_train.shape , X_test.shape , y_train.shape , y_test.shape

((3128, 1225), (783, 1225), (3128, 6), (783, 6))

In [18]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR

model = MultiOutputRegressor(SVR())
model.fit(X_train, y_train)
print(model.score(X_train,y_train))

0.7329411565227023


In [20]:
y_pred_1 = model.predict(X_test)
y_pred_1

array([[2.86058491, 2.87887166, 2.92969528, 2.76360033, 2.77943371,
        2.91579594],
       [3.2261834 , 3.07675643, 3.37935623, 3.2082381 , 3.1302623 ,
        2.94221474],
       [3.44805585, 3.39581105, 3.58457457, 3.52229238, 3.26448109,
        3.27072029],
       ...,
       [3.04928009, 2.89481135, 3.03398788, 2.89202542, 2.98556335,
        2.75591047],
       [3.57147988, 3.17313055, 3.42111813, 3.1578643 , 2.91358643,
        3.44073983],
       [3.08154698, 3.14305952, 3.3366348 , 3.33043312, 3.45280596,
        3.05217526]])

In [21]:
y_test[0]

array([2.5, 3. , 3. , 3. , 2.5, 3. ])

In [23]:
y_pred_1 = np.round(y_pred_1,2)

In [25]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_test , y_pred_1)

0.5211728395061725

In [26]:
from sklearn.linear_model import LinearRegression

model = MultiOutputRegressor(LinearRegression())
model.fit(X_train, y_train)
print(model.score(X_train,y_train))

0.374039543507024


In [27]:
from sklearn.neighbors import KNeighborsRegressor

model = MultiOutputRegressor(KNeighborsRegressor())
model.fit(X_train, y_train)
print(model.score(X_train,y_train))

0.22166152169937514
